In [1]:
#pip install pandas numpy matplotlib seaborn scikit-learn torch torchvision pillow requests tqdm


In [1]:
import os

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import cross_val_score, train_test_split

%matplotlib inline

In [2]:
df = pd.read_csv("test2.csv")


In [3]:
df = df.reset_index(drop=True)

In [4]:
df["idd"] = range(len(df))


In [5]:
df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,idd
0,2591820310,20141006T000000,4,2.25,2070,8893,2.0,0,0,4,...,2070,0,1986,0,98058,47.4388,-122.162,2390,7700,0
1,7974200820,20140821T000000,5,3.00,2900,6730,1.0,0,0,5,...,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283,1
2,7701450110,20140815T000000,4,2.50,3770,10893,2.0,0,2,3,...,3770,0,1997,0,98006,47.5646,-122.129,3710,9685,2
3,9522300010,20150331T000000,3,3.50,4560,14608,2.0,0,2,3,...,4560,0,1990,0,98034,47.6995,-122.228,4050,14226,3
4,9510861140,20140714T000000,3,2.50,2550,5376,2.0,0,0,3,...,2550,0,2004,0,98052,47.6647,-122.083,2250,4050,4


In [6]:
df.shape

(5404, 21)

In [7]:
df.isnull().sum()

id               0
date             0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
idd              0
dtype: int64

In [8]:
df["living_ratio"] = df["sqft_living"] / (df["sqft_living15"] + 1)
df["lot_ratio"] = df["sqft_lot"] / (df["sqft_lot15"] + 1)


In [9]:
df["basement_flag"] = (df["sqft_basement"] > 0).astype(int)
df["basement_frac"] = df["sqft_basement"] / (df["sqft_living"] + 1)


In [10]:
df["house_age"] = 2025 - df["yr_built"]
df["was_renovated"] = (df["yr_renovated"] > 0).astype(int)


In [11]:
from sklearn.cluster import KMeans

coords = df[["lat", "long"]]
df["geo_cluster"] = KMeans(n_clusters=30, random_state=42).fit_predict(coords)


In [13]:
features = [
    "bedrooms",
    "bathrooms",
    "sqft_living",
    "sqft_above",
    "sqft_basement",
    "floors",
    "grade",
    "condition",
    "yr_built",
    "house_age",          
    "basement_flag",    
    "basement_frac",      
    "sqft_living15",
    "sqft_lot15",
    "living_ratio",  
    "lot_ratio" ,
    "view",
    "waterfront",
    "geo_cluster"
]


In [14]:
X = df[features]
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(df[features])

In [15]:
import joblib
import torch
import torch.nn as nn
from torchvision import models, transforms
import numpy as np
import pandas as pd
import os
from PIL import Image



In [16]:
df.isnull().sum()

id               0
date             0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
idd              0
living_ratio     0
lot_ratio        0
basement_flag    0
basement_frac    0
house_age        0
was_renovated    0
geo_cluster      0
dtype: int64

In [17]:
tab_model = joblib.load("lgb_model(R2-0.8948).pkl")
residual_model = joblib.load("image_residual_model.pkl")
scaler = joblib.load("scaler.pkl")


In [18]:
tab_preds_test = tab_model.predict(X)


C:\Users\amitj\anaconda3\envs\cdc-satellite\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [19]:
import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from torchvision import models, transforms
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_squared_error

In [20]:
class ImageOnlyDataset(Dataset):
    def __init__(self, df, image_dir):
        self.df = df.reset_index(drop=True)
        self.image_dir = image_dir

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.loc[idx]
        img_path = os.path.join(self.image_dir, f"{row['idd']}.png")

        assert os.path.exists(img_path), f"Missing image for ID {row['idd']}"

        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)

        return image


In [21]:
device = "cuda" if torch.cuda.is_available() else "cpu"

resnet = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
resnet.fc = nn.Identity()   # output = 512-d embedding
resnet = resnet.to(device)
resnet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [22]:
def extract_embeddings(df, image_dir):
    dataset = ImageOnlyDataset(df, image_dir)
    loader = DataLoader(
        dataset,
        batch_size=32,
        shuffle=False,   
        num_workers=0
    )

    embeddings = []

    with torch.no_grad():
        for imgs in loader:
            imgs = imgs.to(device)
            emb = resnet(imgs)
            embeddings.append(emb.cpu())

    return torch.cat(embeddings).numpy()


In [23]:
X_test_img = extract_embeddings(df, "images/test")
print("Image embeddings:", X_test_img.shape)


Image embeddings: (5404, 512)


In [24]:
image_residuals_test = residual_model.predict(X_test_img)


In [25]:
final_test_prices = tab_preds_test + image_residuals_test


In [26]:
submission = pd.DataFrame({
    "id": df["id"],
    "predicted_price": final_test_prices
})

submission.to_csv("final_test_predictions.csv", index=False)
print("Saved final_test_predictions.csv")

Saved final_test_predictions.csv
